In [43]:
import requests
import json
from datetime import datetime, date
from pprint import pprint
import random
import pandas as pd 


def get_bearer_token():
    url = "https://api.looklook.app/api/auth/login"
    payload = json.dumps({
          "email": "josh+admin@spark-nyc.com",
          "password": "Josh740808!"
          })
    headers = {
          'Content-Type': 'application/json', 
          }
    response = requests.request("POST", url, headers=headers, data=payload)
    response = response.json() 
    token = response['token']
    return token 

bearer = get_bearer_token()
payload={}
headers = {
  'Authorization': bearer,
  'Cookie': 'adminBackdoor=Bearer%20eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySWQiOiIxYzUwMWZmYS0wMDhkLTQyMjItYWUxZC1kOGE2Y2E5NWJhYTYiLCJpYXQiOjE2MTc2MDc5NjQsImV4cCI6MTYzMzE1OTk2NCwiYXVkIjoiYWRtaW4ubG9va2xvb2suYXBwIiwiaXNzIjoiTG9va0xvb2tBcHAiLCJzdWIiOiJsb29rbG9va0BzcGFyay1ueWMuY29tIn0.MOFmAspJ6KCxgNHmm-RYmMldLYoLnzVL3PDyZlLSD69sSwGfHIJC4w8Jpz90sNi7LFbSx-OgHFaV7ogTh8fKLKoCkAsSrw3ixeAJOq6y0pvQdnyBOCf_QfSYZXfiH4HHciNwGzp7WmIgK1z6AVo3mOJL4ThKvhy2pH4mnYC8MPhjVw38EWJ37fZhF0uBu45xQxPKA9oWcxMXZpzGMZhr0eeY3N0CMJzGWJRlg3WqK6Z-7xGAj7us7cB5HvTOdnx-PR8o1BErBY2wtoi-YKqyhP5s3tzUjmc-cm1fxgn1Wd1sIZXi-Dpc-jb1st442W8-DHcFPTqZ13D6Fa6Z9Y4SEw; AWSALB=bP6czL5Ru/rQmphYq36DMvT6DxykC6HTtYkDLG2UV1Pb4VbKmandS5Z+O0mluiBbbavQJH5hdCBZyrGLQKycsgILiXfBpZ4eD/Ev/FP902YxYavMELYU4kJUS6jT; AWSALBCORS=bP6czL5Ru/rQmphYq36DMvT6DxykC6HTtYkDLG2UV1Pb4VbKmandS5Z+O0mluiBbbavQJH5hdCBZyrGLQKycsgILiXfBpZ4eD/Ev/FP902YxYavMELYU4kJUS6jT'
}


def get_list_of_teams():
    url = "https://api.looklook.app/api/teams"
    response = requests.request("GET", url, headers=headers, data=payload)
    response = response.json()
    for i in response['teams']:
        print(f"Name: {i['name']}")
        print(f"ID: {i['id']}")
        print(f"Parent ID: {i['parentId']}\n")


def get_list_of_studies(teamId: int):
    url = f"https://api.looklook.app/api/studies?perPage=100&page=1&teamId={teamId}"
    response = requests.request("GET", url, headers=headers, data=payload)
    json_response = response.json()
    for i in json_response['studies']:
        print(f"Name:\t\t{i['name']}")
        print(f"id:\t\t{i['id']}")
        print(f"Participants:\t{i['meta']['participantCount']}")
        print(f"Topics:\t\t{i['meta']['topicCount']}")
        print(f"Questions:\t{i['meta']['questionCount']}")
        print(f"Groups:\t\t{i['groups']}\n")


def get_study_name(studyId: int) -> str:
    url = f"https://api.looklook.app/api/studies/{studyId}"
    response = requests.request("GET", url, headers=headers, data=payload)
    json_response = response.json()
    study_name = json_response['study']['name']
    return study_name


def get_participants_in_study(studyId: int):
    url = f"https://api.looklook.app/api/studies/{studyId}/participants"
    response = requests.request("GET", url, headers=headers, data=payload)
    json_response = response.json()

    for i in json_response['participants']:
        if i['birthday'] == None:
             age = "None"
        else:
            epoch = int(i['birthday'] / 1000)
            birth_year = date.fromtimestamp(epoch)
            birth_year = int(birth_year.strftime("%Y"))
            today = date.today()
            age = today.year - birth_year

        if i['gender'] == 20:
            gender = "Female"
        else:
            gender = "Male"

        if i['accountParameters']['city'] == None:
            city = "None"
        else:
            city = i['accountParameters']['city']

        if i['accountParameters']['country'] == None:
            country = "None"
        elif i['accountParameters']['country'] == "United States":
              country = "USA"
        elif i['accountParameters']['country'] == "united states":
              country = "USA"
        else:
              country = i['accountParameters']['country']

        if i['accountParameters']['ethnicity'] == None: 
            ethnicity = "None"
        else:
            ethnicity = i['accountParameters']['ethnicity'] 

        if i['group']['name'] == None: 
            group = "None"
        else:
            group = i['group']['name']
        
        if i['lastName'] == None or i['firstName'] == None:
            name = "None"
        else:
            name = i['firstName'] + " " + i['lastName']

        p_id = i['id']   
        email = i['email']

        participant_dict = {
                            "id": p_id,
                            "name": name, 
                            "email": email,
                            "age": age, 
                            "city": city, 
                            "country": country, 
                            "ethnicity": ethnicity,
                            "gender": gender, 
                            "group": group
                            }

        yield participant_dict 


def shorten_ids(column: pd.DataFrame):
    column = column.astype('string')
    new_column = column.str.split("-", 1)
    new_column = [i.pop(0) for i in new_column]
    return new_column


def create_df_from_study(studyId: int) -> pd.DataFrame:
    participants = get_participants_in_study(studyId)
    part_dict = [i for i in participants]
    df = pd.DataFrame(part_dict)
    pd.set_option('display.max_colwidth', 300)
    pd.set_option('display.colheader_justify', 'center')
    pd.options.display.max_columns = 100
    pd.options.display.max_rows = 400 
    df = df.drop(columns='email')
    df['id'] = shorten_ids(df['id'])
    return df